# DL-TFM in Pytorch I

## Imports and technical prerequisites

In [ ]:
from tracNet import TracNet
from data_preparation import matFiles_to_npArray, extract_fields, reshape
from training_and_evaluation import initialize_weights, run_epoch, fit

import copy
import datetime
import gc
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import shapely as sh
import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Tuple
from scipy.io import loadmat, savemat
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

%matplotlib inline

Set seeds for reproducability.

In [ ]:
random_seed = 1
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False

Use CUDA if available.

In [ ]:
gc.collect()
torch.cuda.empty_cache()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Running on device: {device}")

## 1. Data loading and preprocessing

Set paths to training.

In [ ]:
dspl_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/train/trainData104/dspl'
dsplRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/train/trainData104/dsplRadial'
trac_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/train/trainData104/trac'
tracRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/train/trainData104/tracRadial'

Create `ndarrays` of `dicts` containing either the inputs or targets.

In [ ]:
samples = matFiles_to_npArray(dspl_path) # each dict has keys ['brdx', 'brdy', 'dspl', 'name']
dspl_radials = matFiles_to_npArray(dsplRadial_path) # each dict has keys ['dspl', 'name']
targets = matFiles_to_npArray(trac_path) # each dict has keys ['brdx', 'brdy', 'trac', 'name']
trac_radials = matFiles_to_npArray(tracRadial_path) # each dict has keys ['trac', 'name']

Split training data into train and validation set using stratified samples.

In [ ]:
radial_X_train, radial_X_val, radial_y_train, radial_y_val = train_test_split(dspl_radials, trac_radials, test_size=0.05)
X_train, X_val, y_train, y_val = train_test_split(samples, targets, test_size=0.05)
X_train, X_val, y_train, y_val = np.append(radial_X_train, X_train), np.append(radial_X_val, X_val), np.append(radial_y_train, y_train), np.append(radial_y_val, y_val)

Extract displacement and traction fields from the data and drop (meta-) data which is not needed for training purposes.

In [ ]:
X_train = extract_fields(X_train)
X_val = extract_fields(X_val)
y_train = extract_fields(y_train)
y_val = extract_fields(y_val)

Current shape of the datasets is (samples, width, height, depth). 
Reshape them to (samples, channels, depth, height, width) to allow 3D-Convolutions during training.

In [ ]:
X_train = reshape(X_train)
X_val = reshape(X_val)
y_train = reshape(y_train)
y_val = reshape(y_val)

Convert datasets to Pytorch tensors.

In [ ]:
X_train = torch.from_numpy(X_train).double()
X_val = torch.from_numpy(X_val).double()
y_train = torch.from_numpy(y_train).double()
y_val = torch.from_numpy(y_val).double()

Create Pytorch dataloaders and specify batch sizes.

In [ ]:
train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)

batch_size = 4

dataloaders = {}
dataloaders['train'] = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
dataloaders['val'] = DataLoader(val_set, batch_size=2*batch_size, num_workers=8, pin_memory=True)

## 2. Training

Define custom loss function corresponding to the forward loss function in the Matlab regression layer for image-to-image networks:
 
$${loss} = \frac{1}{2} \sum \limits _{p=1} ^{HWC} (t_{p} - y_{p})^{2}$$

In [ ]:
class Custom_Loss(nn.Module):
    def __init__(self):
        super(Custom_Loss, self).__init__();
    
    def forward(self, predictions, target):
        loss = 0.5 * torch.sum(torch.pow(target - predictions, 2))
        return loss

Instantiate the model (including logs for evaluation), the optimizer and start training.

In [ ]:
NAME = "TracNet104-{:%Y-%b-%d %H:%M:%S}".format(datetime.datetime.now())
writer = SummaryWriter(log_dir='logs/{}'.format(NAME))
model = TracNet(n_channels=1).double()
model.to(device)
model.apply(initialize_weights)

# To create a computional graph in Tensorboard, uncomment the following lines.
# inputs, targets = next(iter(dataloaders['train']))
# inputs = inputs.to(device)
# targets = targets.to(device)
# writer.add_graph(model, inputs)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0006, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=10, gamma=0.7943, verbose=True)
loss_fn = Custom_Loss()

fit(model, loss_fn, scheduler, dataloaders, optimizer, device, max_epochs=100, patience=5)

## 3. Evaluation

Specify paths to test data.

In [ ]:
# Test data
test_dspl_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/test/generic/testData104/dspl'
test_trac_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/test/generic/testData104/trac'

Create `ndarrays` of `dicts` containing either the inputs or targets.

In [ ]:
test_inputs = matFiles_to_npArray(test_dspl_path) # each dict has keys ['brdx', 'brdy', 'dspl', 'name']
test_targets = matFiles_to_npArray(test_trac_path)  # each dict has keys ['brdx', 'brdy', 'trac', 'name']

Extract displacement and traction fields from the data to perform testing. As some fields from the original data is needed to compute the error correctly, copy it beforehand.

In [ ]:
X_test = extract_fields(test_inputs)
y_test = extract_fields(test_targets)

Current shape of the datasets is (samples, width, height, depth). Reshape them to (samples, channels, depth, height, width) to make them fit to the TracNet architecture.

In [ ]:
X_test = reshape(X_test)
y_test = reshape(y_test)

Convert datasets to Pytorch tensors.

In [ ]:
X_test = torch.from_numpy(X_test).double()
y_test = torch.from_numpy(y_test).double()

Create Pytorch dataloader and specify batch size.

In [ ]:
test_set = TensorDataset(X_test, y_test)
batch_size = 4
test_dataloader = DataLoader(test_set, batch_size=3*batch_size, shuffle=False, pin_memory=True)

Load model.

In [ ]:
model = TracNet(n_channels=1).double()
model.load_state_dict(torch.load('/home/alexrichard/LRZ Sync+Share/ML in Physics/Models/TracNet104-2022-May-12 19:12:09.pth'), strict=False)
model.eval()

Example: Calculate error between prediction and ground truth of sample 'MLData0040.mat' in the 'generic' folder of the test set.

In [ ]:
prediction = model(X_test)
ground_truth = y_test

The `brdx` and `brdy` values are the same in the displacement field file and the traction field file.

In [ ]:
def errorTrac(dspl_filepath, trac_filepath, plot=False):
    '''
    Calculate error of traction stress field realtive to ground truth as normalized MSE for cell interior only.
    '''
    dspl_file = loadmat(dspl_filepath)  # load displacement field file, including cell coordinates
    brdx = np.array(dspl_file['brdx']) # x-values of predicted cell border
    brdy = np.array(dspl_file['brdy'])  # y-values of predicted cell border
    trac_pred = predictTrac(torch.from_numpy(reshape(np.array(dspl_file['dspl'])[np.newaxis,])).double(), 10) # predict traction field for a single sample
    trac_gt = torch.from_numpy(reshape(np.array(loadmat(trac_filepath)['trac'])[np.newaxis,])).double()
    
    zipped = np.array(list(zip(brdx[0], brdy[0])))  # array with (x,y) pairs of cell border coordinates
    polygon = sh.geometry.Polygon(zipped)  # create polygon

    interior = np.zeros((dspl_file['dspl'].shape[0], dspl_file['dspl'].shape[1]), dtype=int)  # create all zero matrix
    for i in range(len(interior)):  # set all elements in interior matrix to 1 that actually lie within the cell
        for j in range(len(interior[i])):
            point = Point(i, j)
            if polygon.contains(point):
                interior[i][j] = 1

    # plot polygons using geopandas
    if plot:
        p = gpd.GeoSeries(polygon)
        p.plot()
        plt.show()
    
    with torch.no_grad():
        # update prediction and ground truth by discarding areas outside of cell borders
        trac_pred[-1, -1, 0, :, :] = trac_pred[-1, -1, 0, :, :] * torch.from_numpy(interior)
        trac_pred[-1, -1, 1, :, :] = trac_pred[-1, -1, 1, :, :] * torch.from_numpy(interior)
        trac_gt[-1, -1, 0, :, :] = trac_gt[-1, -1, 0, :, :] * torch.from_numpy(interior)
        trac_gt[-1, -1, 1, :, :] = trac_gt[-1, -1, 1, :, :] * torch.from_numpy(interior)

        # compute rmse
        normalization = torch.count_nonzero(torch.from_numpy(interior) * torch.ones(size=interior.shape))
        mse = torch.sum(((trac_pred[-1, -1, 0, :, :] - trac_gt[-1, -1, 0, :, :])**2 + (trac_pred[-1, -1, 1, :, :] - trac_gt[-1, -1, 1, :, :])**2) / normalization)
        rmse = torch.sqrt(mse)
        msm = torch.sum((trac_pred[-1, -1, 0, :, :]**2 + trac_gt[-1, -1, 1, :, :]**2) / normalization)
        rmsm = np.sqrt(msm)
        error = rmse / rmsm

    return error

In [ ]:
def predictTrac(logits, E):
    with torch.no_grad():
        S = logits.size(dim=3)
        mag = S / 104
        conversion = E / (10 * S)
        return model(logits) * conversion

In [ ]:
errorTrac('/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/test/generic/testData104/mini_dspl/MLData0040.mat', '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/test/generic/testData104/mini_trac/MLData0040.mat', plot=True)

Load test file of the Easy-to-use TFM package

In [ ]:
easy_test = loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/Easy-to-use_TFM_package-master/test_data/input_data.mat')

In [ ]:
easy_test_displacement_pos = loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/Easy-to-use_TFM_package-master/test_data/input_data_displacement_pos.mat')['input_data_displacement_pos']

In [ ]:
easy_test_displacement_vec = loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/Easy-to-use_TFM_package-master/test_data/input_data_displacement_vec.mat')['input_data_displacement_vec']

In [ ]:
easy_test_displacement_pos.shape

In [ ]:
easy_test_displacement_vec.shape

In [ ]:
easy_test_displacement_pos

Example: Calculate error between prediction and ground truth of sample 'MLData0040.mat' in the 'generic' folder of the test set.

In [ ]:
prediction = model(X_test)
ground_truth = y_test

The `brdx` and `brdy` values are the same in the displacement field file and the traction field file.

In [ ]:
def errorTrac(dspl_filepath, trac_filepath, plot=False):
    '''
    Calculate error of traction stress field realtive to ground truth as normalized MSE for cell interior only.
    '''
    dspl_file = loadmat(dspl_filepath)  # load displacement field file, including cell coordinates
    brdx = np.array(dspl_file['brdx']) # x-values of predicted cell border
    brdy = np.array(dspl_file['brdy'])  # y-values of predicted cell border
    trac_pred = predictTrac(torch.from_numpy(reshape(np.array(dspl_file['dspl'])[np.newaxis,])).double(), 10) # predict traction field for a single sample
    trac_gt = torch.from_numpy(reshape(np.array(loadmat(trac_filepath)['trac'])[np.newaxis,])).double()
    
    zipped = np.array(list(zip(brdx[0], brdy[0])))  # array with (x,y) pairs of cell border coordinates
    polygon = sh.geometry.Polygon(zipped)  # create polygon

    interior = np.zeros((dspl_file['dspl'].shape[0], dspl_file['dspl'].shape[1]), dtype=int)  # create all zero matrix
    for i in range(len(interior)):  # set all elements in interior matrix to 1 that actually lie within the cell
        for j in range(len(interior[i])):
            point = Point(i, j)
            if polygon.contains(point):
                interior[i][j] = 1

    # plot polygons using geopandas
    if plot:
        p = gpd.GeoSeries(polygon)
        p.plot()
        plt.show()
    
    with torch.no_grad():
        # update prediction and ground truth by discarding areas outside of cell borders
        trac_pred[-1, -1, 0, :, :] = trac_pred[-1, -1, 0, :, :] * torch.from_numpy(interior)
        trac_pred[-1, -1, 1, :, :] = trac_pred[-1, -1, 1, :, :] * torch.from_numpy(interior)
        trac_gt[-1, -1, 0, :, :] = trac_gt[-1, -1, 0, :, :] * torch.from_numpy(interior)
        trac_gt[-1, -1, 1, :, :] = trac_gt[-1, -1, 1, :, :] * torch.from_numpy(interior)

        # compute rmse
        normalization = torch.count_nonzero(torch.from_numpy(interior) * torch.ones(size=interior.shape))
        mse = torch.sum(((trac_pred[-1, -1, 0, :, :] - trac_gt[-1, -1, 0, :, :])**2 + (trac_pred[-1, -1, 1, :, :] - trac_gt[-1, -1, 1, :, :])**2) / normalization)
        rmse = torch.sqrt(mse)
        msm = torch.sum((trac_pred[-1, -1, 0, :, :]**2 + trac_gt[-1, -1, 1, :, :]**2) / normalization)
        rmsm = np.sqrt(msm)
        error = rmse / rmsm

    return error

In [ ]:
def predictTrac(logits, E):
    with torch.no_grad():
        S = logits.size(dim=3)
        mag = S / 104
        conversion = E / (10 * S)
        return model(logits) * conversion

In [ ]:
errorTrac('/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/test/generic/testData104/mini_dspl/MLData0040.mat', '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/test/generic/testData104/mini_trac/MLData0040.mat', plot=True)

Load test file of the Easy-to-use TFM package

In [ ]:
easy_test = loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/Easy-to-use_TFM_package-master/test_data/input_data.mat')

In [ ]:
easy_test_displacement_pos = loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/Easy-to-use_TFM_package-master/test_data/input_data_displacement_pos.mat')['input_data_displacement_pos']

In [ ]:
easy_test_displacement_vec = loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/Easy-to-use_TFM_package-master/test_data/input_data_displacement_vec.mat')['input_data_displacement_vec']

In [ ]:
easy_test_displacement_pos.shape

In [ ]:
easy_test_displacement_vec.shape

In [ ]:
easy_test_displacement_pos

Example predictions

In [ ]:
pred = model(X_test)

In [ ]:
pred[0][0]

In [ ]:
pred[0][0].shape

In [ ]:
pred_1 = (pred[0][0]).detach().numpy()

In [ ]:
pred_1 = np.moveaxis(pred_1, [0, 1, 2], [2, 1, 0])

In [ ]:
pred_1 = savemat('torch_pred_2.mat', {'trac': pred_1})

In [ ]:
matlab_predicted_trac_field = np.array(matlab_prediction['ans'])

In [ ]:
matlab_predicted_trac_field.shape

In [ ]:
matlab_predicted_trac_field = np.array(matlab_prediction['ans'])[np.newaxis, :]
matlab_predicted_trac_field = np.moveaxis(matlab_predicted_trac_field[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
matlab_predicted_trac_field = torch.from_numpy(matlab_predicted_trac_field).double()

In [ ]:
def forward(predictions, target):
    loss = 0.5 * torch.sum(torch.pow(target - predictions, 2))
    return loss

In [ ]:
forward(predicted_trac_field, gt_trac_field)

In [ ]:
forward(matlab_predicted_trac_field, gt_trac_field)

In [ ]:
torch.allclose(predicted_trac_field,matlab_predicted_trac_field, atol=5)

In [ ]:
predicted_trac_field

In [ ]:
matlab_predicted_trac_field